In [1]:
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 43.2 MB/s eta 0:00:00


In [2]:
import geopandas as gpd

In [77]:
from pyproj import Geod
from shapely import wkt
import json

In [27]:
df_borders = gpd.read_file('/content/administrative_borders.geojson')

In [28]:
full_list_of_columns = df_borders.columns.values.tolist()
full_list_of_columns

['id',
 '@id',
 'CEMT',
 'access',
 'addr:city',
 'addr:country',
 'addr:region',
 'admin_level',
 'alt_name',
 'alt_name:ca',
 'alt_name:de',
 'axle_load',
 'barrier',
 'bicycle',
 'boat',
 'boundary',
 'branch',
 'bridge',
 'bridge:name',
 'bus',
 'bus:oneway',
 'busway',
 'check_date:lit',
 'comment',
 'construction',
 'contact:website',
 'covered',
 'cycleway',
 'cycleway:both',
 'cycleway:lane',
 'cycleway:left',
 'cycleway:left:oneway',
 'cycleway:right',
 'description',
 'divider',
 'electrified',
 'fence_type',
 'fixme',
 'foot',
 'footway',
 'frequency',
 'gauge',
 'gvr:code',
 'height',
 'hgv',
 'highspeed',
 'highway',
 'horse',
 'hw',
 'image',
 'incline',
 'int_name',
 'lane_markings',
 'lanes',
 'lanes:backward',
 'lanes:bus',
 'lanes:bus:backward',
 'lanes:bus:forward',
 'lanes:forward',
 'lanes:psv',
 'lanes:psv:backward',
 'lanes:psv:forward',
 'layer',
 'lit',
 'living_street',
 'loading_gauge',
 'loc_name',
 'location',
 'man_made',
 'material',
 'maxheight',
 'maxsp

In [29]:
target_list_of_columns = ['id', 'addr:region', 'boundary', 'name', 'geometry']

In [30]:
drop_list_of_columns = []
for i in full_list_of_columns:
  if i not in target_list_of_columns:
    drop_list_of_columns.append(i)

In [31]:
df_borders.drop(drop_list_of_columns, axis=1, inplace=True)

In [37]:
type_list = []
for i in range(df_borders.shape[0]):
  type_list.append(str(type(df_borders.iloc[i]['geometry'])).split("'")[1])

In [39]:
df_borders['type'] = type_list

In [41]:
df_borders = df_borders[(df_borders['type'] == 'shapely.geometry.polygon.Polygon') | (df_borders['type'] == 'shapely.geometry.multipolygon.MultiPolygon')]

In [ ]:
df_borders.drop('type', axis=1, inplace=True)

In [ ]:
df_borders['addr:region'] = df_borders['addr:region'].fillna('Москва')

In [54]:
df_borders

,id,addr:region,boundary,name,geometry
0,relation/181288,Москва,administrative,район Внуково,"MULTIPOLYGON (((37.19085 55.60184, 37.19088 55..."
1,relation/226927,Москва,administrative,район Кунцево,"MULTIPOLYGON (((37.37864 55.80837, 37.37939 55..."
2,relation/240229,Москва,administrative,район Крылатское,"POLYGON ((37.42256 55.74518, 37.42346 55.74587..."
3,relation/364001,Москва,administrative,Мещанский район,"POLYGON ((37.62144 55.75929, 37.62142 55.75924..."
4,relation/364551,Москва,administrative,район Сокольники,"POLYGON ((37.66909 55.82134, 37.66830 55.82018..."
...,...,...,...,...,...
140,relation/2092927,Москва,administrative,поселение Михайлово-Ярцевское,"POLYGON ((37.10850 55.43951, 37.10650 55.43852..."
141,relation/2092928,Москва,administrative,поселение Роговское,"POLYGON ((36.98644 55.31559, 36.98654 55.31465..."
142,relation/2092929,Москва,administrative,поселение Рязановское,"POLYGON ((37.45463 55.49408, 37.45456 55.49405..."
143,relation/2092931,Москва,administrative,поселение Щаповское,"POLYGON ((37.46539 55.39592, 37.46521 55.39659..."


In [59]:
    def calculate_area(border_value, median_value, df):
        area_list = []
        geod = Geod(ellps="WGS84")
        for i in range(df.shape[0]):
            if str(type(df.iloc[i]['geometry'])).split("'")[1] == 'shapely.geometry.point.Point':
                area = median_value

            else:
                poly = wkt.loads(str(df.iloc[i]['geometry']))
                area = abs(geod.geometry_area_perimeter(poly)[0])

            if area > border_value:
                area = median_value
            area_list.append(area)
        return area_list

In [60]:
border_value = 10000000000000
median_value = 9999999999999

In [63]:
df_borders['area'] = calculate_area(border_value, median_value, df_borders)

/usr/local/lib/python3.9/dist-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [69]:
df_borders.rename(columns={'name': 'district_name'}, inplace=True)

<ipython-input-69-4686bdc778ad>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_borders.rename(columns={'name': 'district_name'}, inplace=True)


In [71]:
df_borders.rename(columns={'area': 'district_area', 'id': 'district_id'}, inplace=True)

<ipython-input-71-9dabe7fe1152>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_borders.rename(columns={'area': 'district_area'}, inplace=True)


In [72]:
df_borders

,id,addr:region,boundary,district_name,geometry,district_area
0,relation/181288,Москва,administrative,район Внуково,"MULTIPOLYGON (((37.19085 55.60184, 37.19088 55...",1.542062e+07
1,relation/226927,Москва,administrative,район Кунцево,"MULTIPOLYGON (((37.37864 55.80837, 37.37939 55...",5.265320e+07
2,relation/240229,Москва,administrative,район Крылатское,"POLYGON ((37.42256 55.74518, 37.42346 55.74587...",1.218994e+07
3,relation/364001,Москва,administrative,Мещанский район,"POLYGON ((37.62144 55.75929, 37.62142 55.75924...",4.515232e+06
4,relation/364551,Москва,administrative,район Сокольники,"POLYGON ((37.66909 55.82134, 37.66830 55.82018...",1.024107e+07
...,...,...,...,...,...,...
140,relation/2092927,Москва,administrative,поселение Михайлово-Ярцевское,"POLYGON ((37.10850 55.43951, 37.10650 55.43852...",6.469836e+07
141,relation/2092928,Москва,administrative,поселение Роговское,"POLYGON ((36.98644 55.31559, 36.98654 55.31465...",1.780844e+08
142,relation/2092929,Москва,administrative,поселение Рязановское,"POLYGON ((37.45463 55.49408, 37.45456 55.49405...",4.064429e+07
143,relation/2092931,Москва,administrative,поселение Щаповское,"POLYGON ((37.46539 55.39592, 37.46521 55.39659...",8.631793e+07


In [88]:
district_map = {}
for i in range(df_borders.shape[0]):
  district_map[df_borders.loc[i]['district_name']] = df_borders.loc[i]['district_id']

In [95]:
with open('district_list.json', 'w') as f:
  json.dump(district_map, f)

In [99]:
with open("district_list.json", "r") as fh:
     k = json.load(fh)

In [74]:
df_borders.to_file('administrative_borders_transform.geojson', driver="GeoJSON")